In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go
import os
from sklearn.metrics import mean_squared_error

# OpenModelica Models in Python

This tutorial explains first how to handle OpenModelica models on python then how to transform them into functions (usefull for CORRAI).

## Physical model
For this example we propose a resistance/capacity approach.
 Based on electrical circuit analogy, each layer of the wall is modeled by two resistance and a capacity:
 
 <img src="images/Wall_model.png"  height="300">

The following is a brief description of the thermal model, as it is not the scope of this document. See the <code>*mo</code> for full informations.

- Each wall layer is modeled by 2 thermal resistances and a capacity.
    - Resistances : $ R_1 = R_2 = \frac{ep_{layer}}{lambda_{layer} \times 2} $
    - Capacity : $ C = ep_{layer} \times rho_{layer} \times cap_{layer} $


- Inside and outside convection/conduction transfers are model as a constant value thermal resistance.


- Infrared transfers are considered :
    - With the sky, with $ T_{sky} = 0.0552T_{ext}^{1.5} $
    - With the surrounding considered to be at $ T_{ext} $


- Short wave solar radiation heat flux is computed $Sw_{gain} = Pyr \times \alpha_{coat} $ with $Pyr$ the measured solar radiation onthe wall (W/m²) and  $\alpha_{coat}$ the coating solar absorbtion coefficient.


- Temperatures $ T_{ext}$ and $T_{int} $ are boundary conditions

## Import the Open Modelica model

#### Load the data
First we load measured data used for the calibration or boundary conditions, suing <code>pandas</code>.

In [ ]:
reference_df = pd.read_csv(
    "C:/Users/thubert/PycharmProjects/modelitool/tutorials/ressources/study_df.csv",
    index_col=0,
    parse_dates=True
)

#### Set simulation options
Initial conditions for the layers temperatures are taken from the measured data.
 It is assumed to be the mean temperature measured by the sensors on each face of a layer.
 In python and using modelica "object name", it can be written :

In [ ]:
init_dict = {
    "Twall_init": 24.81 + 273.15,
    "Tins1_init": 19.70 + 273.15,
    "Tins2_init": 10.56 + 273.15,
    "Tcoat_init": 6.4 + 273.15,
}

We specify the simulation running options. As the initial condition, it is written
as a python dictionary.

In Modelica, <code>startTime</code> and <code>stopTime</code> correspond to the number
of seconds since the beginning of the year. The values can be found in the file created
earlier using <code>df_to_combitimetable</code>

Another way is to use the index of the <code>DataFrame</code> we just created.
The modelitool function <code>modelitool.combitabconvert.datetime_to_seconds</code>
helps you convert datetime index in seconds.

For the sensitivity Analysis we will just use the 2 first day 22/03 and 23/03

<code>stepSize</code> is the simulation timestep size. In this case it's 5min or
300sec.

<code>tolerance</code> and <code>solver</code> are related to solver configuration
do not change if you don't need to.

In [ ]:
from modelitool.combitabconvert import datetime_to_seconds

In [ ]:
from modelitool.combitabconvert import datetime_to_seconds
time_corr = pd.Series(
    datetime_to_seconds(reference_df.index),
    index=reference_df.index
)

In [ ]:
second_index = datetime_to_seconds(reference_df.loc["2018-03-22":"2018-03-23"].index)

simulation_opt = {
        "startTime": second_index[0],
        "stopTime": second_index[-1],
        "stepSize": 300,
        "tolerance": 1e-06,
        "solver": "dassl",
        "outputFormat": "csv"
}

We can now define a modelitool <code>Simulator</code>. This object is designed to handle modelica simulation and output post treatment (it uses the library **OmPython** https://pypi.org/project/pydelica/)

In [ ]:
from modelitool.simulate import Simulator

In [ ]:
# Values in output list correspond to sensors name and value "T"
simu = Simulator(
    model_path=r"C:/Users\thubert\PycharmProjects\tutorials\resources/etics_v0.mo",
    simulation_options=simulation_opt,
    simulation_path=Path(r"C:\Users\thubert\Documents\Modelica\temp"),
    init_parameters=init_dict,
    boundary_df=reference_df,
    output_list=["T_coat_ins.T",
                 "T_ins_ins.T",
                 "Tw_out.T"],
    lmodel=["Modelica"]
)

You can set values to parameters using method <code>set_param_dict</code>

In [ ]:
# These parameters have been identified in a previous study.
simu.set_param_dict({
    'IR_Amb.Gr': 0.0631,
    'IR_sky.Gr': 0.0466,
})

#### Run the simulation

From here, it is very simple to run a simulation using <code>simulate()</code>
method, and to get the results required in <code>output_list</code> using
<code>get_results()</code> method.
Simflags similar to flags in OpenModelica can be specified.

In [ ]:
simu.simulate(simflags="-noEventEmit -noRestart")

#### Results

In [ ]:
results = simu.get_results()
results

*Important Note: Temperature outputs of the modelica model are in Kelvin.
We manually add 273.15°C to compare results or to compare measurementand model outputs*

In [ ]:
# Plotly lines
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results.index,
    y=results["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Model_results"
))

fig.add_trace(go.Scatter(
    x=results.index,
    y=reference_df.T_Ins_Ins + 273.15,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Model vs. Reality : temperature between two layers of insulation',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

## Corrai connector
Various methods used in other libraries (optimisation, calibration, etc.) might require functions as model inputs. We created the class <code>ModelicaFunction</code> that defines a function based on a Modelitool Simulator.

In [ ]:
from modelitool.corrai_connector import ModelicaFunction

The class <code>ModelicaFunction</code> requires several arguments: 
- a simulator, which is the fully configured Modelitool Simulator object we defined earlier
- a list of parameters
- a list of indicators (or objectives)

Let's first define the parameters using enumerators from class <code>Parameter</code>:interval of possible values and initial value.
The names must correspond to Modelica syntax

In [ ]:
from corrai.base.parameter import Parameter

In [ ]:
id_params = [   
    {
        Parameter.NAME: 'Lambda_ins.k',
        Parameter.INTERVAL: (0.04*0.4, 0.04*1.6),
        Parameter.INIT_VALUE: 0.0,
        Parameter.TYPE: "Real"
    }, 
    {    
        Parameter.NAME: 'Alpha_clo.k',
        Parameter.INTERVAL: (0.2, 0.95),
        Parameter.INIT_VALUE: 0.0,
        Parameter.TYPE: "Real"
    }, 
]

Then, we choose two temperatures as our indicators:
- a temperature directly calculated in the OpenModelica Models (specified as an output in the Simulator) 
- a custom temperature "T_moy_wall".

In [ ]:
indicators = ["T_ins_ins.T", "T_moy_wall"]

As "T_moy_wall" is not a direct output of the modelica model, its calculation should be specified in a seperate dictionnary, giving parameters used for its calculation and the related function.

In [ ]:
def mean_and_offset(x, y):
    return (x + y)/2 + 0.02 

custom_ind_dict = {
    "T_moy_wall": {
        "depends_on": ["T_ins_ins.T", "Tw_out.T"],
        "function": mean_and_offset
    }
}

 A reference dictionary can be specified if any of the indicators is to be compared to existing data using an aggregation method. 

In [ ]:
reference_dict = {
    "T_ins_ins.T": "T_Ins_Ins"
}

The used aggregation methods are _**mean**_ by default. If a reference dictionnary is used, you should specify an aggregation methods using predicted and ground truth data, such as : 
- Normalise mean biased error
- Coefficient of variation of the root mean squared error between measured and modeled data

In [ ]:
from modelitool.metrics import  nmbe
from modelitool.metrics import cv_rmse

agg_methods_dict = {
    "T_ins_ins.T": cv_rmse,
}

Finally, we create the python object mf using <code>ModelicaFunction</code>. This is the object that can be used in CORRAI.

In [ ]:
training_set = reference_df[["T_Ins_Ins"]].loc["2018-03-22":"2018-03-23"] + 273.15

mf = ModelicaFunction(
    simulator=simu,
    param_list=id_params,
    agg_methods_dict=agg_methods_dict,
    indicators=indicators,
    reference_df=training_set,
    reference_dict=reference_dict,
    custom_ind_dict = custom_ind_dict
)

## Application 
Let's see an application using CORRAI library, for an optimization. Let's define a mono objective problem : minimize the cv_rmse on "T_ins_ins.T" optmizing the values of $\lambda_{etics}$ and $\alpha_{coating}$

In [ ]:
indicators_application = ["T_ins_ins.T"]

mf_app = ModelicaFunction(
        simulator=simu,
        param_list=id_params,
        agg_methods_dict=agg_methods_dict,
        indicators=indicators_application,
        reference_df=training_set,
        reference_dict=reference_dict,
)

from corrai.multi_optimize import MyProblem

problem = MyProblem(
    parameters=id_params,
    obj_func_list=[mf_app],
    func_list=[],
    function_names=indicators_application,
    constraint_names=[],
)

We choose here a genetic algorithm for a single-objective problem from pymoo (https://pymoo.org/algorithms/soo/ga.html). It can be easily customized with different evolutionary operators and applies to a broad category of problems.

In [ ]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.problems import get_problem
from pymoo.optimize import minimize

algorithm = GA(
    pop_size=20,
    eliminate_duplicates=True
)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Let's run the simulation with these values.

In [ ]:
simu.set_param_dict({
    'Lambda_ins.k': res.X[0], # 0.064
    'Alpha_clo.k': res.X[1], # 0.95
})

In [ ]:
simu.simulate(simflags="-noEventEmit -noRestart")
results_optim = simu.get_results()

#### Results after optimization

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results.index,
    y=results["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='red',
    name="First_results"
))

fig.add_trace(go.Scatter(
    x=results_optim.index,
    y=results_optim["T_ins_ins.T"],
    fill=None,
    mode='lines',
    line_color='brown',
    name="Optim_results"
))

fig.add_trace(go.Scatter(
    x=results.index,
    y=reference_df.T_Ins_Ins + 273.15,
    fill='tonexty', 
    mode='lines',
    line_color='orange',
    name="Reference_measure"
))

fig.update_layout(
    title='Optimization vs. Reality : temperature between two layers of insulation',
    xaxis_title='Date',
    yaxis_title='Temperature [K]')

fig.show()

The results are not satisfying :
- The insulation heat conductivity is set to 0.064, the maximum allowed value. This is very unlikely.
- The solar radiation heat gain $\alpha_{coating}$ is set to the maximum allowed value 0.95
whereas the true value should be around 0.7.
The algorithm tried to reproduce the high temperatures by maximizing the solar heat gain and the heat transfers
through the first layer of insulation, leading to probably false results

There could be a problem linked to the model heat capacity that have not been considered here.

In this case, we succeeded at creating a model that more or less reproduce the wall thermal behavior.
Unfortunatly we cannot draw any conclusion on the wall material properties. That's a shame as it was the whole point
of the experiment.

Several imporvements can be made to this test bed :
- Perform additionnal measurements such as heat flux
- Provide a more accurate model (better description of phenomenon). But be carefull, this may required more accurate measure such as wind speed and direction.

Finally other identification method (and much more complex and computationnaly intensive) may be used such as stochastic method based on Baye's theorem. This can be performed using CORRAI library, and in future will be described in a tutorial.